In [13]:
import pandas as pd


In [12]:
#download data
battling_data = pd.read_csv('data/2022 MLB Player Stats - Batting.csv', encoding='latin_1')
pitching_data = pd.read_csv('data/2022 MLB Player Stats - Pitching.csv', encoding='latin_1')
exit_velocity_data = pd.read_csv('data/mlb-batter-exit-velocity.csv', encoding='latin_1')


In [14]:
battling_data

,Rk;Name;Age;Tm;Lg;G;PA;AB;R;H;2B;3B;HR;RBI;SB;CS;BB;SO;BA;OBP;SLG;OPS;OPS+;TB;GDP;HBP;SH;SF;IBB
0,1;CJ Abrams*;21;TOT;NL;90;302;284;33;70;12;2;2...
1,2;CJ Abrams*;21;SDP;NL;46;139;125;16;29;5;0;2;...
2,3;CJ Abrams*;21;WSN;NL;44;163;159;17;41;7;2;0;...
3,4;Albert Abreu;26;TOT;AL;1;0;0;0;0;0;0;0;0;0;0...
4,5;Albert Abreu;26;KCR;AL;1;0;0;0;0;0;0;0;0;0;0...
...,...
987,988;Bradley Zimmer*;29;TOT;AL;100;101;89;14;9;...
988,989;Bradley Zimmer*;29;TOR;AL;77;87;76;11;8;4;...
989,990;Bradley Zimmer*;29;PHI;NL;9;16;16;4;4;1;0;...
990,991;Bradley Zimmer*;29;TOR;AL;23;14;13;3;1;0;0...


In [5]:
pitching_data

,Rk;Name;Age;Tm;Lg;W;L;W-L%;ERA;G;GS;GF;CG;SHO;SV;IP;H;R;ER;HR;BB;IBB;SO;HBP;BK;WP;BF;ERA+;FIP;WHIP;H9;HR9;BB9;SO9;SO/W
0,1;Cory Abbott;26;WSN;NL;0;5;0;5.25;16;9;7;0;0;...
1,2;Albert Abreu;26;TOT;AL;2;2;0.5;3.26;33;0;11;...
2,3;Albert Abreu;26;TEX;AL;0;0;0;3.12;7;0;0;0;0;...
3,4;Albert Abreu;26;KCR;AL;0;0;0;4.15;4;0;3;0;0;...
4,5;Albert Abreu;26;NYY;AL;2;2;0.5;3.16;22;0;8;0...
...,...
1076,1077;Rob Zastryzny*;30;NYM;NL;0;0;0;9;1;0;0;0;...
1077,1078;Rob Zastryzny*;30;LAA;AL;0;0;0;6;5;0;1;0;...
1078,1079;Angel Zerpa*;22;KCR;AL;2;1;0.667;1.64;3;2...
1079,1080;T.J. Zeuch;26;CIN;NL;0;3;0;15.19;3;3;0;0;...


In [6]:
exit_velocity_data

,id,rank,year,player,batted_ball_events,launch_angle,sweet_spot_percentage,max_ev,average_ev,fly_ball_line_drive_ev,ground_ball_ev,max_distance,average_distance,average_homerun,hard_hit_95mph+,hard_hit_percentage,hard_hit_swing_percentage,total_barrels,barrels_batted_balls_percentage,barrels_plate_appearance_percentage
0,1,1,2022,"Judge, Aaron",341,14.6,37.8,118.4,95.8,100.2,89.1,465,205,413.0,210,61.6,20.3,91,26.7,15.8
1,2,2,2022,"Alvarez, Yordan",304,12.1,40.1,117.4,95.5,98.2,92.7,469,193,403.0,186,61.2,23.7,59,19.4,12.7
2,3,3,2022,"Trout, Mike",234,24.7,37.2,114.4,91.7,94.9,87.7,472,218,407.0,120,51.3,16.5,46,19.7,11.6
3,4,4,2022,"Schwarber, Kyle",312,18.9,34.6,114.8,93.3,99.7,87.2,468,197,415.0,170,54.5,17.6,64,20.5,11.5
4,5,5,2022,"Stanton, Giancarlo",228,10.2,26.3,119.8,94.6,98.2,94.1,445,160,400.0,117,51.3,17.8,42,18.4,11.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1978,1979,246,2015,"PeÃ±a, Brayan",302,8.0,24.8,104.9,85.6,86.9,85.8,378,137,NaN,52,17.2,8.9,1,0.3,0.3
1979,1980,247,2015,"Reyes, JosÃ©",431,13.3,28.1,108.4,84.6,86.0,85.9,419,151,386.0,80,18.6,8.8,1,0.2,0.2
1980,1981,248,2015,"Sogard, Eric",326,11.6,32.2,107.5,85.5,86.5,85.9,410,155,410.0,61,18.7,10.1,1,0.3,0.2
1981,1982,249,2015,"Revere, Ben",536,5.1,34.9,111.4,83.1,86.0,81.3,389,133,367.0,78,14.6,8.2,0,0.0,0.0
